In [7]:
import sympy as sp
import orbipy as op

In [455]:
import math
import numpy as np

In [9]:
model = op.crtbp3_model()

In [532]:
class CT:
    def __init__(self, model):
        self.c2 = self.c(2)
        self.mu = model.mu
        self.g = 1.-model.mu - model.L1
        self.w1 = sp.sqrt((self.c2 - 2 - sp.sqrt(9*self.c2**2 - 8*self.c2))/(-2))
        self.w2 = sp.sqrt(self.c2)
        self.l1 = sp.sqrt((self.c2 - 2 + sp.sqrt(9*self.c2**2 - 8*self.c2))/2)
        self.s1 = sp.sqrt(2*self.l1*((4 + 3*self.c2)*self.l1**2 + 4 + 5*self.c2 - 6*self.c2**2))
        self.s2 = sp.sqrt(self.w1*((4 + 3*self.c2)*self.w1**2 - 4 - 5*self.c2 + 6*self.c2**2))
    
    def c(self, n):
        g = sp.Symbol('g')
        mu = sp.Symbol('mu')
        return (mu+((-1)**n)*((1 - mu)*g**(n + 1))/((1 - g)**(n + 1)))/g**3

    def h(self, n):
        if n<=2:
            raise RuntimeError('n must be > 2')
        x,y,z = sp.symbols('x y z')
        sq = sp.sqrt(x**2+y**2+z**2)
        return self.c(n)*sq**n*sp.together(sp.legendre(n, x/sq))
    
    def R(self):
        return sp.Matrix([[2*self.l1/self.s1,0,0,-2*self.l1/self.s1, 2*self.w1/self.s2,0],
                      [(self.l1**2-2*self.c2-1)/self.s1,(-self.w1**2-2*self.c2-1)/self.s2,0,(self.l1**2-2*self.c2-1)/self.s1,0,0],
                      [0,0,1/sp.sqrt(self.w2),0,0,0],
                      [(self.l1**2+2*self.c2+1)/self.s1,(-self.w1**2+2*self.c2+1)/self.s2,0,(self.l1**2+2*self.c2+1)/self.s1,0,0],
                      [(self.l1**3+(1-2*self.c2)*self.l1)/self.s1,0,0,(-self.l1**3-(1-2*self.c2)*self.l1)/self.s1,(-self.w1**3+(1-2*self.c2)*self.w1)/self.s2,0],
                      [0,0,0,0,0,sp.sqrt(self.w2)]]).subs({'g': ct.g, 'mu':ct.mu}).evalf()
    
    def symp_change(self):
        x,y,z,px,py,pz = sp.symbols('x1 y1 z1 px1 py1 pz1')
        mat = sp.Matrix([[x],[y],[z],[px],[py],[pz]])
        return self.R()*mat
    
    def h_symp(self, n):
        x, y,z, px, py, pz = sp.symbols('x y z px py pz')
        change = self.symp_change()
        h = self.h(n)
        h = h.subs({'x': change[0], 'y': change[1], 'z': change[2]})
        h = h.subs({'x1': x, 'y1': y, 'z1': z, 'px1': px, 'py1': py, 'pz1': pz})
        h = h.subs({'g': ct.g, 'mu':ct.mu}).expand().evalf()
        return h
    
    def h_complex(self, n):
        y1,z1,py1,pz1 = sp.symbols('y1 z1 py1 pz1')
        y,z,py,pz = sp.symbols('y z py pz')
        sq2 = math.sqrt(2)
        y_change = (y1 + sp.I*py1)/sq2
        z_change = (z1 + sp.I*pz1)/sq2
        py_change = (py1 + sp.I*y1)/sq2
        pz_change = (pz1 + sp.I*z1)/sq2
        if n == 2:
            h = self.h2_symp()
        elif n>2:
            h = self.h_symp(n)
        else:
            raise RuntimeError('unsupported n')
        h = h.subs({'y': y_change, 'z': z_change, 'py': py_change, 'pz': pz_change}).expand()
        h = h.subs({'y1': y, 'z1': z, 'py1': py, 'pz1': pz})
        return h #self.chop(h)
    
    def gen_func(self, h_comp):
        x, y,z,px,py,pz = sp.symbols('x y z px py pz')
        n1 = self.l1.subs({'g': ct.g, 'mu':ct.mu}).evalf()
        n2 = sp.I*self.w1.subs({'g': ct.g, 'mu':ct.mu}).evalf()
        n3 = sp.I*self.w2.subs({'g': ct.g, 'mu':ct.mu}).evalf()
        pol = sp.Poly(h_comp, x,y,z,px,py,pz)
        mons = pol.monoms()
        gen = 0
        for mon in mons:
            a1 = (mon[3]-mon[0])
            a2 = (mon[4]-mon[1])
            a3 = (mon[5]-mon[2])
            if not (a1==0 and a2==0 and a3==0):
                denominator = a1*n1 + a2*n2 + a3*n3
                sym_part = x**mon[0]*y**mon[1]*z**mon[2]*px**mon[3]*py**mon[4]*pz**mon[5]
                coef = -1*pol.coeff_monomial(mon)
                gen += coef*sym_part/denominator
        return gen.expand()
    
    def pbracket(self, f, g):
        x, y,z, px, py, pz = sp.symbols('x y z px py pz')
        q = [x ,y ,z]
        p = [px, py, pz]
        res = 0
        for i in range(3):
            res += sp.diff(f, q[i])*sp.diff(g, p[i]) - sp.diff(f, p[i])*sp.diff(g, q[i])
        return res.expand()
    
    def h2(self):
        x, y,z, px, py, pz = sp.symbols('x y z px py pz')
        h = (self.c2*(-2*x**2 + y**2 + z**2) + 2*y*px - 2*x*py + px**2 + py**2 + pz**2)/2
        return h
    
    def h2_symp(self):
        change = self.symp_change()
        h = self.h2().subs({'x': change[0], 'y': change[1], 'z': change[2], 'px': change[3], 'py': change[4], 'pz': change[5]})
        x, y,z, px, py, pz = sp.symbols('x y z px py pz')
        h = h.subs({'x1': x, 'y1': y, 'z1': z, 'px1': px, 'py1': py, 'pz1': pz})
        h = h.subs({'g': ct.g, 'mu':ct.mu}).expand()
        return h #self.chop(h)
    
    def chop(self, h):
        x, y,z,px,py,pz = sp.symbols('x y z px py pz')
        pol = sp.Poly(h, x,y,z,px,py,pz)
        mons = pol.monoms()
        h_new = 0
#         import pdb; pdb.set_trace()
        for mon in mons:
            coef = pol.coeff_monomial(mon)
            coef_chopped = self.chop_coef(coef)
            a, b = coef_chopped.as_real_imag()
            if abs(a)+abs(b) > 0:
                sym_part = x**mon[0]*y**mon[1]*z**mon[2]*px**mon[3]*py**mon[4]*pz**mon[5]
                h_new += coef_chopped*sym_part
        
        return h_new
    
    def chop_coef(self, coef):
        a, b = coef.as_real_imag()
        new_coef = self.chop_num(a) + self.chop_num(b)*sp.I
#         print('Old coef: {}; New coef: {}'.format(coef, new_coef))
        return new_coef

    def chop_num(self, num, tol=1e-14):
        if abs(num) > tol:
            return num
        else:
            return 0
        
    def new_var(self, var, g, n):
        new_var = 0
        prev = 0
        new_var += var
        prev += new_var
        for i in np.arange(1, n+1):
            cur = self.pbracket(prev, g)
            new_var += cur/math.factorial(i)
            prev = cur.copy()
            
        return new_var.expand()#self.realify(new_var)
    
    
    def realify(self, expr):
        y1,z1,py1,pz1 = sp.symbols('y1 z1 py1 pz1')
        y,z,py,pz = sp.symbols('y z py pz')
        sq2 = math.sqrt(2)
        y_change = (y1 - sp.I*py1)/sq2
        z_change = (z1 - sp.I*pz1)/sq2
        py_change = (py1 - sp.I*y1)/sq2
        pz_change = (pz1 - sp.I*z1)/sq2
        
        real_expr = expr.subs({'y': y_change, 'z': z_change, 'py': py_change, 'pz': pz_change})
        real_expr = real_expr.subs({'y1': y, 'z1': z, 'py1': py, 'pz1': pz})
        
        return real_expr.expand()

In [638]:
def print_coef_rules(h):
    x, y,z,px,py,pz = sp.symbols('x y z px py pz')
    pol = sp.Poly(h, x,y,z,px,py,pz)
    mons = pol.monoms()
    print(len(mons))
    for i, mon in enumerate(mons):
        print("{}: {} -> {}".format(i, mon, pol.coeff_monomial(mon)))

In [533]:
ct = CT(model)

Оргинальный гамильтониан $H = H_2 + \sum_{i=3}^\infty H_i$ 

$H_2$-мономы второй степени оригинального полинома (в комплексном виде)

In [ ]:
h2c = ct.h_complex(2)

$H_3$-мономы третьей степени оригинального полинома (в комплексном виде)

In [ ]:
h3c = ct.h_complex(3)

$G_3$ - производящая функция (в комплексном виде)

In [ ]:
g3 = ct.gen_func(h3c)

Новый гамильтониан $\hat{H} = H + \{ H,G_3\} + \frac{1}{2!} \{\{ H,G_3\} ,G_3\} + \dots$

$\hat{H}_2 = H_2$

$\hat{H}_3 = H_3 + \{ H_2,G_3\}$

$\hat{H}_4 = H_4 + \{ H_3,G_3\} + \frac{1}{2}\{\{ H_2,G_3\} , G_3\}$

$\hat{H}_3$=

In [604]:
h2g3 = ct.pbracket(h2c, g3).expand()
res = ct.chop((h3c + h2g3).simplify())
res

0

$\hat{H}_4$ = 

In [535]:
h3g3 = ct.pbracket(h3c, g3)
h2g3g3 = ct.pbracket(h2g3,g3)

h24=ct.h_complex(4)+h3g3+(h2g3g3)/2
h24 = ct.chop(h24.expand())

$G_4$ - производящая функция

In [537]:
g4 = ct.gen_func(h24)

Новый гамильтониан $\tilde{H} = \hat{H} + \{ \hat{H},G_4\} + \frac{1}{2!} \{\{ \hat{H},G_4\} ,G_4\} + \dots$

$\tilde{H}_2 = H_2$

$\tilde{H}_3 = 0$

$\tilde{H}_4 = \hat{H}_4 + \{ \hat{H}_2,G_4\} =$

In [538]:
res2 = ct.chop((h24+ct.pbracket(h2c, g4)).expand())
res2

1.32193627173593*px**2*x**2 + 3.22427953091787*I*px*py*x*y + 2.87855762692204*I*px*pz*x*z - 0.844084910107101*py**2*y**2 - 0.859677804016423*py*pz*y*z - 0.758573690186595*pz**2*z**2

Остались мономы, в которых все степени одинаковые

$\tilde{H}_5 = \hat{H}_5 $

$\hat{H} = H + \{ H,G_3\} + \frac{1}{2!} \{\{ H,G_3\} ,G_3\} + \dots$

$\hat{H}_5 = H_5 + \{ H_4,G_3\} + \frac{1}{2!} \{\{ H_3,G_3\} ,G_3\} +\frac{1}{3!} \{  \{\{ H_2,G_3\} ,G_3\},G_3\}=$

In [610]:
h5_hat = ct.h_complex(5) + ct.pbracket(ct.h_complex(4), g3) + ct.pbracket(h3g3, g3)/2 + ct.pbracket(h2g3g3, g3)/6
h5_hat = h5_hat.expand()

$G_5=$

In [612]:
g5 = ct.gen_func(h5_hat)

$\bar{H} = \tilde{H} + \{ \tilde{H},G_5\} + \frac{1}{2!} \{\{ \tilde{H},G_5\} ,G_5\} + \dots$

$\bar{H}_5 = \tilde{H}_5 +  \{ \tilde{H}_2, G_5\}=$

In [615]:
res3 = ct.chop(h5_hat + ct.pbracket(h2c, g5))
res3

0

Формула для вычисления компоненты x - соответствующей гамильтониану $\hat{H}$

In [605]:
g3_real = ct.realify(g3)
x_new_real = ct.chop(ct.new_var(x, g3_real, 2))
x_new_real

0.0264277499984091*px**2*py + 0.0815992412872559*px**2*x + 0.0162001404123427*px**2*y + 0.0493329683360396*px**2 + 0.0282943570534649*px*py**2 + 0.0231047637023565*px*py*x + 0.0449980539210955*px*py*y - 0.199849945064169*px*py + 0.0281692481928682*px*pz**2 + 0.0251324283189624*px*pz*z + 0.0677359012833492*px*x**2 + 0.0568719480412226*px*x*y - 0.592139176911852*px*x + 0.0713187108395275*px*y**2 - 0.175006884837044*px*y + 0.0737972645121678*px*z**2 - 0.00608776120416747*py**3 + 0.0498985877435698*py**2*x + 0.0267593656742168*py**2*y - 0.0761397580092677*py**2 - 0.00245538232581553*py*pz**2 - 0.00130392957760074*py*pz*z + 0.0791077011471059*py*x**2 + 0.108118036725567*py*x*y - 0.00280322423674001*py*y**2 - 0.196537185400899*py*y - 0.0733286406739247*py*z**2 + 0.0442069661278257*pz**2*x + 0.0278935396858513*pz**2*y - 0.132618513495058*pz**2 + 0.0609003205062315*pz*x*z + 0.0589789537060891*pz*y*z - 0.166669663845512*pz*z + 0.0815992412872558*x**3 + 0.05843014027954*x**2*y - 0.29606958845592

Формула для вычисления компоненты x - соответствующей гамильтониану $\tilde{H}$

In [609]:
g4_real = ct.realify(g4)
x_new_real2 = ct.new_var(x_new_real, g4_real, 2)
x_new_real2 = ct.chop(x_new_real2)
x_new_real2

In [653]:
# print_coef_rules(x_new_real2)

Формула для вычисления компоненты x - соответствующей гамильтониану $\bar{H}$

In [634]:
g5_real = ct.chop(ct.realify(g5))

In [645]:
x_new_real3 = ct.new_var(x_new_real2, g5_real, 1)

In [646]:
x_new_real3 = ct.chop(x_new_real3)

In [654]:
# print_coef_rules(x_new_real3)

Аналогично для px

In [592]:
px_new_real = ct.chop(ct.new_var(px, g3_real, 2))

In [593]:
px_new_real2 = ct.new_var(px_new_real, g4_real, 2)
px_new_real2 = ct.chop(px_new_real2)

In [650]:
px_new_real3 = ct.new_var(px_new_real2, g5_real, 1)
px_new_real3 = ct.chop(px_new_real3)

Сохранение

In [549]:
from sympy.utilities.lambdify import lambdify

In [629]:
x, y,z,px,py,pz = sp.symbols('x y z px py pz')
f3 = lambdify([x,y,z,px,py,pz], x_new_real, modules='numpy')
f4 = lambdify([x,y,z,px,py,pz], x_new_real2, modules='numpy')

In [648]:
f5 = lambdify([x,y,z,px,py,pz], x_new_real3, modules='numpy')

In [583]:
import dill
dill.settings['recurse'] = True

In [607]:
dill.dump(f3, open("x3.bin", "wb"))

In [608]:
dill.dump(f4, open("x4.bin", "wb"))

In [649]:
dill.dump(f5, open("x5.bin", "wb"))

In [594]:
px3 = lambdify([x,y,z,px,py,pz], px_new_real, modules='numpy')
px4 = lambdify([x,y,z,px,py,pz], px_new_real2, modules='numpy')

In [651]:
px5 = lambdify([x,y,z,px,py,pz], px_new_real3, modules='numpy')

In [595]:
dill.dump(px3, open("px3.bin", "wb"))
dill.dump(px4, open("px4.bin", "wb"))

In [652]:
dill.dump(px5, open("px5.bin", "wb"))